# **Проект исследования вакансий Junior аналитиков с сайта LinkedIn**

## **Цель исследования**

Визуализировать информацию о рынке вакансий для аналитиков в Европе, в разрезе:
- по стране и по типу занятости
- общее количество вакансий
- нанимающих компаний с указанием количества вакансий,
- ТОП 10 сфер деятельности компаний, которые нанимают аналитиков
- размер компаний и количество вакансий
- требуемые хард скилы
- Дополнительно (необязательно) - зависимость количества кандидатов на вакансию от даты публикации


### **Описание  данных:**

Предоставлен датасет с сырыми данными после парсинга сайта LinkedIn, который необходимо обработать и создать датасет, содержащий:
- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию

### **План исследования**

Работать будем в следующем порядке:
- обработка исходного файла и выделение в нем требуемых признаков
- проверка наличия дубликатов
- проверка наличия нерелевантных вакансий
- удаление ненужных атрибутов
- создание дашборда

In [ ]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
from urllib.parse import urlencode 
import re
import requests
from calendar import day_name
import plotly
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline

In [ ]:
from bs4 import BeautifulSoup
import html5lib

In [ ]:
# установим неограниченную ширину колонок датафреймов
pd.set_option('display.max_colwidth', None)

## **Импорт данных**

In [ ]:
# используя конструкцию try-ecxept для возможности открыть локально и с внешнего источника
try:
    with open(r'C:\Users\ezhmui04\Documents\Поиск работы\Тестовые задания\masterskaya_parsing_LinkedIn_2023_05_23.csv', mode='r', encoding='utf-8',
              errors=None, newline=None, closefd=True, opener=None) as fp:
        raw_data = fp.read()

except:

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/d/gsL7aCpUvlysoQ'  # Сюда вписываете вашу ссылку

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его в текущем директории
    download_response = requests.get(download_url)
    with open('masterskaya_parsing_LinkedIn_2023_05_23.csv', 'wb') as fp:
        fp.write(download_response.content)
        
    # и записываем из него данные в переменную сырых данных
    with open(r'masterskaya_parsing_LinkedIn_2023_05_23.csv', mode='r', encoding='utf-8',
        errors=None, newline=None, closefd=True, opener=None) as fp:
        raw_data = fp.read()

In [ ]:
# используя конструкцию try-ecxept для возможности открыть локально и с внешнего источника
# для загрузки сразу в датафрейм
try:
    raw_df = pd.read_csv(r'C:\Users\ezhmui04\Documents\Поиск работы\Тестовые задания\masterskaya_parsing_LinkedIn_2023_05_23.csv')
    
except:

    # используем api
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/d/gsL7aCpUvlysoQ'
    # получаем url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # загружаем файл в df
    download_response = requests.get(download_url)
    raw_df = pd.read_csv(download_url)

## **Применение "супа" разными парсерами**

In [ ]:
# пробуем распарсить данные разными парсерами
soup = BeautifulSoup(raw_data, 'lxml')

In [ ]:
for job in soup.find_all('h2'):
    print(job.text)

In [ ]:
# пробуем распарсить данные разными парсерами
soup_html = BeautifulSoup(raw_data, 'html.parser')

In [ ]:
for job in soup_html.find_all('h2'):
    print(job.text)

In [ ]:
# пробуем распарсить данные разными парсерами
soup_html5 = BeautifulSoup(raw_data, 'html5lib')

In [ ]:
for job in soup_html5.find_all('h2'):
    print(job.text)

## **Страницы в виде датафрейма**

In [ ]:
raw_df.head(1)

In [ ]:
raw_df.info()

У нас сформировался датафрейм из двух колонок, в первой идет порядковый номер, во второй - содержимое одной страницы

In [ ]:
# запишем в переменную значение одной страницы, что бы полее подробно его изучить
soup_test = BeautifulSoup(raw_df.iloc[0]['html'], 'lxml')

## **Рассмотрение одного блока с вакансией на странице**

In [ ]:
# и посмотрим на него в понятном виде
print(soup_test.prettify())

In [ ]:

job = soup_test.find_all('h2', class_ = 't-24 t-bold jobs-unified-top-card__job-title')

In [ ]:
job = soup_test.find('h2', class_ = 't-24 t-bold jobs-unified-top-card__job-title')

In [ ]:
print(job.text)

# **Общий вывод исследования**